In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import cross_val_score
import numpy as np

In [2]:
model=RandomForestClassifier()

In [3]:
dataset=pd.read_csv("diabetes.csv")

In [4]:
dataset

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                       0.351   31        0  
2                       0.672   32        1  
3                       0.167   21        0  
4                       2.288   33        1  
..                        ...  ...      ...  
763                     0.171   63        0  
764                     0.340   27        0  
765                     0.245   30        0  
766                     0.349   47        1  
767                     0.315   23        0  

[768 rows x 9 columns]

In [5]:
dataset.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [6]:
features=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']

In [7]:
scores = cross_val_score(model, dataset[features], dataset["Outcome"], cv=5)
scores

array([0.72727273, 0.74675325, 0.76623377, 0.81045752, 0.73202614])

In [8]:
scores = cross_val_score(RandomForestClassifier(n_estimators = 2), dataset[features], dataset["Outcome"], cv=5)
scores

array([0.74675325, 0.68831169, 0.72077922, 0.74509804, 0.7124183 ])

In [9]:
np.mean(scores)

0.7226720991426874

In [10]:
mydict={"n_estimators":100,"max_depth":1}

In [11]:
def get_fitness(model,parameters):
    our_model=model(**parameters)
    return np.mean(cross_val_score(our_model, dataset[features], dataset["Outcome"], cv=4))

In [12]:
get_fitness(RandomForestClassifier,mydict)

0.6757812499999999

In [13]:
import random as rnd
#my_range={"n_estimators":(100,500),"max_depth":(1,30)}
def mutate(mydict):
    key= rnd.choice(list(mydict.keys()))
    if (isinstance(mydict[key],float)):
        guess=mydict[key]+ 0.05*rnd.uniform(1,3)
    else:
        guess= int(mydict[key]+ rnd.uniform(1,3))

    mydict[key] = guess
    return mydict
mutate({'n_estimators': 100, 'ccp_alpha':0.1, 'max_depth': 12})

{'n_estimators': 101, 'ccp_alpha': 0.1, 'max_depth': 12}

In [14]:
def create_population(parameters, number):
    population = []
    for i in range(number):
        my_dict = {}
        for parameter in parameters:
            value = parameters[parameter]
            if type(value) == float:
                value *= rnd.uniform(0,2)
                my_dict[parameter] = value
            else:
                value = int(value*rnd.uniform(0,3))
                my_dict[parameter] = value+1
        population.append(my_dict)
    return population
#create_population({'n_estimators' : 100, 'ccp_alpha' : 0.02})

In [15]:
def roulette_selection(fitness_values, model):
    total = sum(fitness_values)
    ranks = [fitness/total for fitness in fitness_values]
    selected_parents = rnd.choices([i for i in range(len(population))], weights = ranks, k = 2) 
    return selected_parents

In [16]:
population = create_population({'n_estimators' : 100, 'ccp_alpha' : 0.02})
fitness_values = [get_fitness(model, child) for child in population]

TypeError: create_population() missing 1 required positional argument: 'number'

In [ ]:
parents = roulette_selection(fitness_values, RandomForestClassifier)

In [ ]:
population = create_population({'min_samples_leaf':2,'min_samples_split':2,'n_estimators' : 100, 'ccp_alpha' : 0.02, 'max_depth':10}, 20)

In [ ]:
population[3]

In [ ]:
get_fitness(model, population[3])

In [18]:
def crossover(parent1, parent2):
    index = rnd.choice(list(parent1.keys()))
    parent1[index], parent2[index] = parent2[index], parent1[index]
    return parent1, parent2

crossover({"n_estimators":100,"max_depth":1},{"n_estimators":140,"max_depth":2})

({'n_estimators': 100, 'max_depth': 2}, {'n_estimators': 140, 'max_depth': 1})

In [20]:
model = RandomForestClassifier
population = create_population({'min_samples_leaf':5,'n_estimators' : 100, 'ccp_alpha' : 0.02, 'max_depth':10}, 100)
print('Calculating Fitness values!')
fitness_values = [get_fitness(model, child) for child in population]
for i in range(1000):
    print('Doing roulette selection, round', i)
    #roulette selection - probability of selection is proportional to fitness score
    parents = roulette_selection(fitness_values, model)
    print('Doing crossover, round', i)
    #crossover
    #population[parents[0]],population[parents[1]]= crossover(population[parents[0]], population[parents[1]])
    crossover0, crossover1 = crossover(population[parents[0]], population[parents[1]])
    fitness_score_parent0=get_fitness(model, crossover0)
    fitness_score_parent1=get_fitness(model, crossover1)
    if fitness_values[parents[0]]<fitness_score_parent0:
        population[parents[0]] = crossover0
        fitness_values[parents[0]] = fitness_score_parent0
    if fitness_values[parents[1]]<fitness_score_parent1:
        population[parents[1]] = crossover1
        fitness_values[parents[1]] = fitness_score_parent1
    #fitness_values[parents[0]], fitness_values[parents[1]] = fitness_score_parent0, fitness_score_parent1
    print('Doing mutation, round', i)
    #mutation - accepted only if it makes population better
    to_be_mutated = rnd.randint(0,len(population)-1)
    child = population[to_be_mutated]
    child_fitness = get_fitness(RandomForestClassifier,child)
    mutation = mutate(child)
    mutation_fitness=get_fitness(RandomForestClassifier,mutation)        
    if mutation_fitness>child_fitness:
        population[to_be_mutated] = mutation
        fitness_values[to_be_mutated] = mutation_fitness
    max_score = max(fitness_values)
    print('Max fitness score for now is', max_score, 'for parameters', population[fitness_values.index(max_score)])

Calculating Fitness values!
Doing roulette selection, round 0
Doing crossover, round 0
Doing mutation, round 0
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 181, 'ccp_alpha': 0.0017563326982396933, 'max_depth': 6}
Doing roulette selection, round 1
Doing crossover, round 1
Doing mutation, round 1
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 181, 'ccp_alpha': 0.0017563326982396933, 'max_depth': 6}
Doing roulette selection, round 2
Doing crossover, round 2
Doing mutation, round 2
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 181, 'ccp_alpha': 0.0017563326982396933, 'max_depth': 6}
Doing roulette selection, round 3
Doing crossover, round 3
Doing mutation, round 3
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 181, 'ccp_alpha': 0.0017563326982396933, 'max_depth': 6}


Doing mutation, round 33
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 3, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 15}
Doing roulette selection, round 34
Doing crossover, round 34
Doing mutation, round 34
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 3, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 35
Doing crossover, round 35
Doing mutation, round 35
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 3, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 36
Doing crossover, round 36
Doing mutation, round 36
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 3, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 37
Doing crossover, round 37
Doing mu

Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 67
Doing crossover, round 67
Doing mutation, round 67
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 68
Doing crossover, round 68
Doing mutation, round 68
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 69
Doing crossover, round 69
Doing mutation, round 69
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 70
Doing crossover, round 70
Doing mutation, round 70
Max fitn

Doing mutation, round 100
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 101
Doing crossover, round 101
Doing mutation, round 101
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 102
Doing crossover, round 102
Doing mutation, round 102
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 103
Doing crossover, round 103
Doing mutation, round 103
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 104
Doing crossover, round 

Doing mutation, round 133
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 134
Doing crossover, round 134
Doing mutation, round 134
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 135
Doing crossover, round 135
Doing mutation, round 135
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 136
Doing crossover, round 136
Doing mutation, round 136
Max fitness score for now is 0.7786458333333334 for parameters {'min_samples_leaf': 4, 'n_estimators': 187, 'ccp_alpha': 0.00015939317761104466, 'max_depth': 30}
Doing roulette selection, round 137
Doing crossover, round 

...

Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 166, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 845
Doing crossover, round 845
Doing mutation, round 845
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 166, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 846
Doing crossover, round 846
Doing mutation, round 846
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 166, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 847
Doing crossover, round 847
Doing mutation, round 847
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 166, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 848
Doing crossover, round 848
Doing mutation, round 848
Max fitn

Doing mutation, round 878
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 879
Doing crossover, round 879
Doing mutation, round 879
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 880
Doing crossover, round 880
Doing mutation, round 880
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 881
Doing crossover, round 881
Doing mutation, round 881
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 882
Doing crossover, round 882
Doing mutati

Doing mutation, round 912
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 913
Doing crossover, round 913
Doing mutation, round 913
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 914
Doing crossover, round 914
Doing mutation, round 914
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 915
Doing crossover, round 915
Doing mutation, round 915
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 916
Doing crossover, round 916
Doing mutati

Doing mutation, round 946
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 947
Doing crossover, round 947
Doing mutation, round 947
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 948
Doing crossover, round 948
Doing mutation, round 948
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 949
Doing crossover, round 949
Doing mutation, round 949
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.38114200627036954, 'max_depth': 32}
Doing roulette selection, round 950
Doing crossover, round 950
Doing mutati

Doing mutation, round 980
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.47892632641076005, 'max_depth': 32}
Doing roulette selection, round 981
Doing crossover, round 981
Doing mutation, round 981
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.47892632641076005, 'max_depth': 32}
Doing roulette selection, round 982
Doing crossover, round 982
Doing mutation, round 982
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.47892632641076005, 'max_depth': 32}
Doing roulette selection, round 983
Doing crossover, round 983
Doing mutation, round 983
Max fitness score for now is 0.7747395833333334 for parameters {'min_samples_leaf': 5, 'n_estimators': 54, 'ccp_alpha': 0.47892632641076005, 'max_depth': 32}
Doing roulette selection, round 984
Doing crossover, round 984
Doing mutati